In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [ ]:
import vso.util
import vso.data
import vso.phot

OBJ_NAME='SZ Lyn'
SESSION_TAG='2025/20250127'
# OBJ_NAME='RZ Lyr'
# SESSION_TAG='20240827'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
settings = vso.util.Settings(session_layout.settings_file_path)
provider = vso.phot.BatchDataProvider(session_layout)


In [ ]:
from astropy.table import QTable, Column, join
import numpy as np
from scipy import stats as sst
from vsopy.util import ValErrDtype

def exposure_regression(session_layout):
    images = QTable.read(session_layout.images_file_path)
    measured = QTable.read(session_layout.measured_file_path)
    sequence = QTable.read(session_layout.sequence_file_path)
    data = join(images, measured, 'image_id')['filter', 'exposure', 'auid', 'peak'].group_by(
        ['filter', 'exposure', 'auid']).groups.aggregate(np.mean)
    data.rename_column('filter', 'band')
    r = join(data, sequence, ['auid', 'band']).group_by(['band', 'exposure'])
    rr = [sst.linregress(10*np.log10(batch['peak']), batch['M']['mag']) for batch in r.groups]
    t = r.groups.keys
    t['Max M'] = Column([(x.intercept, x.intercept_stderr) for x in rr], dtype=ValErrDtype)
    t['mag/dB'] = Column([(x.slope, x.stderr) for x in rr], dtype=ValErrDtype)
    return t

exposure_regression(session_layout)

In [ ]:

from astropy.table import vstack
sessions = [('SZ Lyn', '2025/20250127'), ('SS Aur', '2025/20250127'), ('SA20', '2024/20241003'),
            ('SA38', '2024/20241003'), ('SA41', '2024/20241003'), ('SA111', '2024/20240828'),
            ('SA110', '2024/20240825')]

data = vstack([exposure_regression(layout.get_session(vso.util.Session(tag=s, name=n))) for n, s in sessions])

In [ ]:
d = data[data['band'] == 'V']
d

In [ ]:

import matplotlib.pyplot as plt

d = data[data['band'] == 'Ic']

fig = plt.figure(figsize=(8.00, 6.00))
gs = fig.add_gridspec(1, 1)

ax = fig.add_subplot(gs[0, 0])
ax.errorbar(d['exposure'],  d['Max M']['val'], yerr=d['Max M']['err'], fmt='.')
ax.set_xlabel('Exp')
ax.set_ylabel('Max M')

plt.show()